In [2]:

import os
import argparse
import datetime
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from tutorial_dataset import load_data, load_tif_data
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict

import torch
from torch import nn
from torch import functional as F
# ======================= SSF =======================
def _set_submodule(model, target, module) :
    if target == "":
        raise ValueError("Cannot set the submodule without a target name!")

    atoms = target.split(".")
    name = atoms.pop(-1)
    mod = model

    for item in atoms:
        if not hasattr(mod, item):
            raise AttributeError(
                mod._get_name() + " has no attribute `" + item + "`"
            )

        mod = getattr(mod, item)

        # Use isinstance instead of type here to also handle subclass of nn.Module
        if not isinstance(mod, torch.nn.Module):
            raise AttributeError("`" + item + "` is not an nn.Module")

    setattr(mod, name, module)

def init_ssf_scale_shift(dim):
    scale = nn.Parameter(torch.ones(dim))
    shift = nn.Parameter(torch.zeros(dim))

    nn.init.normal_(scale, mean=1, std=.02)
    nn.init.normal_(shift, std=.02)

    return scale, shift

def ssf_ada(x, scale, shift):
    assert scale.shape == shift.shape
    if x.shape[-1] == scale.shape[0]:
        return x * scale + shift
    elif x.shape[1] == scale.shape[0]:
        return x * scale.view(1, -1, 1, 1) + shift.view(1, -1, 1, 1)
    else:
        raise ValueError('the input tensor shape does not match the shape of the scale factor.')

class Conv2d(nn.Module):
    def __init__(self, module):
        self.__dict__.update(module.__dict__)
        self.ssf_scale, self.ssf_shift = init_ssf_scale_shift(self.out_channels)
    
    def _conv_forward(self, input, weight, bias):
        if self.padding_mode != "zeros":
            return F.conv2d(
                F.pad(
                    input, self._reversed_padding_repeated_twice, mode=self.padding_mode
                ),
                weight,
                bias,
                self.stride,
                _pair(0),
                self.dilation,
                self.groups,
            )
        return F.conv2d(
            input, weight, bias, self.stride, self.padding, self.dilation, self.groups
        )

    def forward(self, x):
        x = self._conv_forward(x, self.weight, self.bias)
        return ssf_ada(x, self.ssf_scale, self.ssf_shift)

class Linear(nn.Module):
    def __init__(self, module):
        self.__dict__.update(module.__dict__)
        self.ssf_scale, self.ssf_shift = init_ssf_scale_shift(self.out_features)
    
    def forward(self, x):
        x = F.linear(x, self.weight, self.bias)
        return ssf_ada(x, self.ssf_scale, self.ssf_shift)

class GroupNorm(nn.Module):
    def __init__(self, module):
        self.__dict__.update(module.__dict__)
        self.ssf_scale, self.ssf_shift = init_ssf_scale_shift(self.num_channels)
    
    def forward(self, x):
        x =  F.group_norm(x, self.num_groups, self.weight, self.bias, self.eps)
        return ssf_ada(x, self.ssf_scale, self.ssf_shift)

class LayerNorm(nn.Module):
    def __init__(self, module):
        self.__dict__.update(module.__dict__)
        self.ssf_scale, self.ssf_shift = init_ssf_scale_shift(self.normalized_shape)
    
    def forward(self, x):
        x =  F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        return ssf_ada(x, self.ssf_scale, self.ssf_shift)

def model_add_ssf(model):
    key_list = [key for key, _ in model.named_parameters()]
    key_set = set()

    for key in key_list:
        i = key.rfind(".")
        key_set.add(key[:i])

    for key in key_set:
        module = model.get_submodule(key)
        
        if isinstance(module, nn.Conv2d):
            # model.set_submodule(key, Conv2d(target))
            _set_submodule(model, key, Conv2d(module))
        elif isinstance(module, nn.Linear):
            _set_submodule(model, key, Linear(module))
        elif isinstance(module, nn.GroupNorm):
            _set_submodule(model, key, GroupNorm(module))
        elif isinstance(module, nn.LayerNorm):
            _set_submodule(model, key, LayerNorm(module))

    # 冻结非ssf参数
    for n, p in model.named_parameters():
        p.requires_grad = "ssf" in n

def get_training_state_dict(model):
    training_state_dict = dict()
    for n, p in model.named_parameters():
        # if "ssf" in n:
        if p.requires_grad:
            training_state_dict.update({n: p})
    
    return training_state_dict
# ======================= SSF =======================

# ======================= RLRR =======================
class RLRRLinear(nn.Module):
    def __init__(self, module):
        self.__dict__.update(module.__dict__)
        self.has_bias = self.bias is not None
        self.scale_col = nn.Parameter(torch.empty(1, self.in_features), requires_grad=True)
        self.scale_line = nn.Parameter(torch.empty(self.out_features, 1), requires_grad=True)
        self.shift_bias = nn.Parameter(torch.empty(1, self.out_features), requires_grad=True)

        nn.init.kaiming_uniform_(self.scale_col)
        nn.init.kaiming_uniform_(self.scale_line)
        nn.init.zeros_(self.shift_bias)

    def forward(self, x):       
        weight = (self.weight * self.scale_col * self.scale_line) + self.weight
        bias = (self.bias + self.shift_bias) if self.has_bias else self.shift_bias
        return F.linear(x, weight, bias)

class RLRRConv(nn.Module):
    def __init__(self, module):
        self.__dict__.update(module.__dict__)
        self.has_bias = self.bias is not None
        self.k = self.kernel_size[0]
        self.scale_col = nn.Parameter(torch.empty(1, self.in_channels * self.k), requires_grad=True)
        self.scale_line = nn.Parameter(torch.empty(self.out_channels * self.k, 1), requires_grad=True)
        if self.has_bias:
            self.shift_bias = nn.Parameter(torch.empty(self.out_channels), requires_grad=True)
            nn.init.zeros_(self.shift_bias)

        nn.init.kaiming_uniform_(self.scale_col)
        nn.init.kaiming_uniform_(self.scale_line)

    def __convert(self, weight):
        weight = weight.permute(0, 2, 1, 3).reshape(self.out_channels * self.k, self.in_channels * self.k)
        weight = self.weight + self.__deconvert(weight * self.scale_line * self.scale_col)
        return weight
    
    def __deconvert(self, weight):
        return weight.reshape(self.out_channels, self.k, self.in_channels, self.k).permute(0, 2, 1, 3)

    def forward(self, x):
        weight = self.__convert(self.weight)
        bias = None
        if self.has_bias:
            bias = self.bias + self.shift_bias
        return F.conv2d(x, weight, bias, stride=self.stride, padding=self.padding)

def model_add_rlrr(model):
    # 冻结原始参数
    model.requires_grad_(False)

    target_module = ["attn1", "ff.net", "proj_in", "proj_out"
                     "in_layers", "emb_layers", "out_layers"]
    key_list = [key for key, _ in model.named_modules()]

    for key in key_list:
        for tk in target_module:
            if tk in key:
                module = model.get_submodule(key)
                if isinstance(module, nn.Conv2d):
                    _set_submodule(model, key, RLRRConv(module))
                elif isinstance(module, nn.Linear):
                    _set_submodule(model, key, RLRRLinear(module))
# ======================= RLRR =======================

In [2]:
# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model(r"./models/cldm_v15_sar2opt.yaml")
model.load_state_dict(load_state_dict("./models/control_sd15_ini.ckpt", location='cpu'), strict=False)
# model.learning_rate = learning_rate
# model.sd_locked = sd_locked
# model.only_mid_control = only_mid_control

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v15_sar2opt.yaml]


f:\crli\ControlNet-v1-1-nightly\cldm\model.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = get_state_dict(torch.load(ckpt_path, map_location=torch.device(l

Loaded state_dict from [./models/control_sd15_ini.ckpt]


<All keys matched successfully>

In [ ]:
clip = model.cond_stage_model
batch_encoding = clip.tokenizer([""]*8, truncation=True, max_length=77, return_length=True,
                                    return_overflowing_tokens=False, padding="max_length", return_tensors="pt")
tokens = batch_encoding["input_ids"]
# tokens
outputs = clip.transformer(input_ids=tokens, output_hidden_states=clip.layer=="hidden")
# outputs
z = outputs.last_hidden_state
# z.repeat(2, 1, 1)
z


In [42]:
import numpy as np

np.save("./clip_prompt_embed.npy", z)

In [ ]:
model_add_rlrr(model)
# model_add_ssf(model)

In [ ]:
model.control_model.requires_grad_(True)
pass

In [ ]:
for n, p in model.model.diffusion_model.named_parameters():
    print(n, p.requires_grad)

In [3]:
import os
import random
import skimage.io as io
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

class FinetuneData(Dataset):
    def __init__(self, gt_dir, cond_dir, cond_images, size=256):
        self.gt_dir = gt_dir
        self.cond_dir = cond_dir
        self.gt_list = os.listdir(gt_dir)
        self.cond_list = cond_images
        self.size = size
        self.pair= {
         "S1_20220719":      "S2_20220719",    
         "S1_20220907":      "S2_20220907",    
         "S1_20220921":      "S2_20220922",    
         "S1_20221019":      "S2_20221018",    
         "S1_20221028":      "S2_20221028",    
         "S1_20221031":      "S2_20221101",    
         "S1_20221103":      "S2_20221103",    
         "S1_20221104":      "S2_20221104",    
         "S1_20221112":      "S2_20221113",    
         "S1_20221125":      "S2_20221125",    
         "S1_20221129":      "S2_20221128",    
         "S1_20221218_18":   "S2_20221218_18",
         "S1_20221218_19":   "S2_20221218_19",
         "S1_20221222":      "S2_20221221",    
         "S1_20221230":      "S2_20221231",    
        }
    def __len__(self):
        return len(self.cond_list)
    def __getitem__(self, idx):
        cond_name = self.cond_list[idx]
        cond = io.imread(os.path.join(self.cond_dir, cond_name))
        gt_name = self.pair[cond_name.split("-")[0]] + "-" + cond_name.split("-")[1]
        gt = io.imread(os.path.join(self.gt_dir, gt_name))
        gt = gt[:, :, :3][:, :, ::-1]
        return dict(image=self.transform(gt), 
                    hint=self.transform(cond), 
                    txt='',
                    gt_name=gt_name,
                    cond_name=cond_name)
    
    def transform(self, image):
        # image = image.transpose(2, 0, 1).astype("float32") / 5000 - 1
        image = self.optimized_linear(image)
        image = image.transpose(2, 0, 1).astype("float32") / 127.5 - 1
        return image

    @staticmethod
    def percent_linear(arr, percent=2):
        arr_min, arr_max = np.percentile(arr, (percent, 100-percent))
        arr = (arr - arr_min) / (arr_max - arr_min) * 255
        arr = np.clip(arr, 0, 255)
        return np.uint8(arr)
    
    @staticmethod
    def optimized_linear(arr):
        a, b = np.percentile(arr, (2.5, 99))
        c = a - 0.1 * (b - a)
        d = b + 0.5 * (b - a)
        arr = (arr - c) / (d - c) * 255
        arr = np.clip(arr, 0, 255)
        return np.uint8(arr)

class DataModule(pl.LightningDataModule):    
    def __init__(self, gt_dir, cond_dir, batch_size=4, num_workers=0, val_size=0.15, size=256):
        super().__init__()
        self.gt_dir = gt_dir
        self.cond_dir = cond_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.val_size = val_size
        self.size = size
        self.setup('fit')
    def setup(self, stage):
        all_images = sorted([u for u in os.listdir(self.cond_dir)])
        random.seed(42)
        random.shuffle(all_images)
        train_size = int((1-self.val_size)*len(all_images))
        train_images = all_images[:train_size]
        val_images = all_images[train_size:]
        test_size = int(0.3*len(val_images))
        test_images = val_images[:test_size]
        val_images = val_images[test_size:]
        self.train_ds = FinetuneData(self.gt_dir, self.cond_dir, train_images, self.size)
        self.val_ds = FinetuneData(self.gt_dir, self.cond_dir, val_images, self.size)
        self.test_ds = FinetuneData(self.gt_dir, self.cond_dir, test_images, self.size)
        print(f"Train size: {len(self.train_ds)}, Val size: {len(self.val_ds)}, Test size: {len(self.test_ds)}")
        print(f"Total size: {len(self.train_ds)+len(self.val_ds)+len(self.test_ds)}")
    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

gt_dir = r"D:\crli\Data\GRD\S2_256"
cond_dir = r"D:\crli\Data\GRD\S1_256"
dataModule = DataModule(gt_dir, cond_dir, batch_size=32, val_size=0)

Train size: 6525, Val size: 0, Test size: 0
Total size: 6525


In [3]:
import numpy as np
import blobfile as bf
import tifffile as tiff
from PIL import Image
from torch.utils.data import Dataset, DataLoader

def _list_image_files_recursively(data_dir):
    results = []
    for entry in sorted(bf.listdir(data_dir)):
        full_path = bf.join(data_dir, entry)
        ext = entry.split(".")[-1]
        if "." in entry and ext.lower() in ["jpg", "jpeg", "png", "gif", "tif"]:
            results.append(full_path)
        elif bf.isdir(full_path):
            results.extend(_list_image_files_recursively(full_path))
    return results

def load_sen12_data(    
    cond_dir,
    gt_dir,
):
    print("cond_dir: ", cond_dir)
    print("gt_dir: ", gt_dir)
    if not cond_dir or not gt_dir:
        raise ValueError("unspecified data directory")

    cond_dir = _list_image_files_recursively(cond_dir)
    gt_dir = _list_image_files_recursively(gt_dir)

    dataset = SEN12Dataset(cond_dir, gt_dir,)
    return dataset 

class SEN12Dataset(Dataset):
    def __init__(
        self,
        cond_dir,
        gt_dir,
        shard=0,
        num_shards=1,
    ):
        super().__init__()
        self.local_images_input = cond_dir[shard:][::num_shards]
        self.local_images_gt = gt_dir[shard:][::num_shards]

    def __len__(self):
        return len(self.local_images_input)

    def __getitem__(self, idx):
        path_input = self.local_images_input[idx]
        path_gt = self.local_images_gt[idx]

        with bf.BlobFile(path_input, "rb") as f:
            pil_image_sar = Image.open(f)
            pil_image_sar.load()
        pil_image_sar = pil_image_sar.convert("RGB")
        with bf.BlobFile(path_gt, "rb") as f:
            pil_image_opt = Image.open(f)
            pil_image_opt.load()
        pil_image_opt = pil_image_opt.convert("RGB")
        
        # arr_input = tiff.imread(path_input) / 127.5 - 1
        # arr_gt = tiff.imread(path_gt) / 127.5 - 1
        arr_input = self.transform(np.array(pil_image_sar))
        arr_gt = self.transform(np.array(pil_image_opt))

        arr_input = arr_input.astype(np.float32)
        arr_gt = arr_gt.astype(np.float32)
        return dict(
            image=arr_gt,
            txt='',
            hint=arr_input,
            gt_name=path_gt.split("\\")[-1]
        )

    def transform(self, image):
        return image.transpose(2, 0, 1).astype("float32") / 127.5 - 1

class SEN12DataModule():    
    def __init__(self, gt_dir, cond_dir, batch_size=4, num_workers=0, val_size=0.15, size=256):
        super().__init__()
        self.gt_dir = gt_dir
        self.cond_dir = cond_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.val_size = val_size
        self.size = size
        self.setup('fit')

    def setup(self, stage):
        all_images = sorted([u for u in os.listdir(self.cond_dir)])
        random.seed(42)
        random.shuffle(all_images)
        train_size = int((1-self.val_size)*len(all_images))
        train_images = all_images[:train_size]
        val_images = all_images[train_size:]
        test_size = int(0.3*len(val_images))
        test_images = val_images[:test_size]
        val_images = val_images[test_size:]
        self.train_ds = SEN12Dataset(self.gt_dir, self.cond_dir, train_images, self.size)
        self.val_ds = SEN12Dataset(self.gt_dir, self.cond_dir, val_images, self.size)
        self.test_ds = SEN12Dataset(self.gt_dir, self.cond_dir, test_images, self.size)
        print(f"Train size: {len(self.train_ds)}, Val size: {len(self.val_ds)}, Test size: {len(self.test_ds)}")
        print(f"Total size: {len(self.train_ds)+len(self.val_ds)+len(self.test_ds)}")

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

train_ds = load_data(r"D:\crli\Data\SEN12_agri\s1", r"D:\crli\Data\SEN12_agri\s2")
train_dataloader = DataLoader(train_ds, batch_size=256, shuffle=False, num_workers=0)

TypeError: load_data() missing 1 required positional argument: 'image_size'

In [ ]:
import torch
from torch.nn import functional as F
from torchmetrics.image.ssim import SSIM
from torchmetrics.image.psnr import PSNR

preds = torch.cat([vae.decode(z).cpu()])
target = torch.cat([gt[0].unsqueeze(0)])
print(target.shape, preds.shape) 
# (bs, C, H, W)
ssim = SSIM()
psnr = PSNR()
print("SSIM:", ssim(preds, target))
print("PSNR:", psnr(preds, target))
print("RMSE:", torch.sqrt(F.mse_loss(preds, target)))

In [6]:
import os
import numpy as np
from tqdm.auto import tqdm

# train_dataloader = dataModule.train_dataloader()
# val_dataloader = dataModule.val_dataloader()

vae = model.first_stage_model.to("cuda")
out_dir = r"D:\crli\Data\SEN12_agri\s2_kl-f8_latents"
os.makedirs(out_dir, exist_ok=True)

for batch in tqdm(train_dataloader):
    gt = batch["image"].to("cuda")
    gt_name = batch["gt_name"]
    # print(gt.shape)

    latents = vae.encode(gt).sample() * 0.18215  # (bsz, 4, /8, /8)
    latents = latents.cpu().numpy()
    # break
    
    for i, z in enumerate(latents):
        np.save(os.path.join(out_dir, gt_name[i].replace("png", "npy")), z)

  0%|          | 0/16 [00:00<?, ?it/s]

In [22]:
import os
import random
import skimage.io as io
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

class FinetuneData(Dataset):
    def __init__(self, gt_dir, cond_dir, cond_images, size=256):
        self.gt_dir = gt_dir
        self.cond_dir = cond_dir
        self.gt_list = os.listdir(gt_dir)
        self.cond_list = cond_images
        self.size = size
        self.pair= {
         "S1_20220719":      "S2_20220719",    
         "S1_20220907":      "S2_20220907",    
         "S1_20220921":      "S2_20220922",    
         "S1_20221019":      "S2_20221018",    
         "S1_20221028":      "S2_20221028",    
         "S1_20221031":      "S2_20221101",    
         "S1_20221103":      "S2_20221103",    
         "S1_20221104":      "S2_20221104",    
         "S1_20221112":      "S2_20221113",    
         "S1_20221125":      "S2_20221125",    
         "S1_20221129":      "S2_20221128",    
         "S1_20221218_18":   "S2_20221218_18",
         "S1_20221218_19":   "S2_20221218_19",
         "S1_20221222":      "S2_20221221",    
         "S1_20221230":      "S2_20221231",    
        }
    def __len__(self):
        return len(self.cond_list)
    def __getitem__(self, idx):
        cond_name = self.cond_list[idx]
        cond = io.imread(os.path.join(self.cond_dir, cond_name))
        gt_name = self.pair[cond_name.split("-")[0]] + "-" + cond_name.split("-")[1].replace("tif", "npy")
        gt = np.load(os.path.join(self.gt_dir, gt_name))
        return dict(image=gt, hint=self.transform(cond), txt='', 
        gt_name=gt_name, cond_name=cond_name)
    
    def transform(self, image):
        image = image.transpose(2, 0, 1).astype("float32") / 5000 - 1
        return image

class DataModule(pl.LightningDataModule):    
    def __init__(self, gt_dir, cond_dir, batch_size=4, num_workers=0, val_size=0.15, size=256):
        super().__init__()
        self.gt_dir = gt_dir
        self.cond_dir = cond_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.val_size = val_size
        self.size = size
        self.setup('fit')
    def setup(self, stage):
        all_images = sorted([u for u in os.listdir(self.cond_dir)])
        random.seed(42)
        random.shuffle(all_images)
        train_size = int((1-self.val_size)*len(all_images))
        train_images = all_images[:train_size]
        val_images = all_images[train_size:]
        test_size = int(0.3*len(val_images))
        test_images = val_images[:test_size]
        val_images = val_images[test_size:]
        self.train_ds = FinetuneData(self.gt_dir, self.cond_dir, train_images, self.size)
        self.val_ds = FinetuneData(self.gt_dir, self.cond_dir, val_images, self.size)
        self.test_ds = FinetuneData(self.gt_dir, self.cond_dir, test_images, self.size)
        print(f"Train size: {len(self.train_ds)}, Val size: {len(self.val_ds)}, Test size: {len(self.test_ds)}")
        print(f"Total size: {len(self.train_ds)+len(self.val_ds)+len(self.test_ds)}")
    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

gt_dir = r"D:\crli\Data\GRD\S2_256_kl-f8_latents"
cond_dir = r"D:\crli\Data\GRD\S1_256"
dataModule = DataModule(gt_dir, cond_dir, batch_size=1)

Train size: 6077, Val size: 752, Test size: 321
Total size: 7150


In [23]:
import numpy as np
from tqdm.auto import tqdm

train_dataloader = dataModule.train_dataloader()
val_dataloader = dataModule.val_dataloader()

for batch in tqdm(train_dataloader):
    gt = batch["image"]
    cond = batch["hint"]
    txt = batch["txt"]
    gt_name = batch["gt_name"]
    cond_name = batch["cond_name"]
    print(gt_name, cond_name)
    break
# print(gt.shape, gt_name)

  0%|          | 0/6077 [00:00<?, ?it/s]

['S2_20221028-498.npy'] ['S1_20221028-498.tif']


In [28]:
import torch
import numpy as np
from pytorch_lightning import seed_everything
seed_everything(0)

pred = torch.randn(4, 3, 256, 256)
target = torch.randn(4, 3, 256, 256)

bsz = 4
num_timesteps = 1000
timesteps = torch.randint(0, num_timesteps, (bsz,)).long()
# snr = self.alphas_cumprod / (1 - self.alphas_cumprod)

linear_start=1e-4
linear_end=2e-2
betas = (torch.linspace(linear_start ** 0.5, linear_end ** 0.5, 1000, dtype=torch.float64) ** 2).numpy()
alphas = 1. - betas
alphas_cumprod = np.cumprod(alphas, axis=0)
snr = torch.from_numpy(alphas_cumprod / (1 - alphas_cumprod))

maybe_clipped_snr = snr.clone()
min_snr_gamma = 5
maybe_clipped_snr.clamp_(max=min_snr_gamma)
loss_weight = maybe_clipped_snr / snr

def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

loss = torch.nn.functional.mse_loss(pred, target, reduction='none')

# loss = reduce(loss, 'b ... -> b', 'mean')
extract(loss_weight, timesteps, loss.shape)

Global seed set to 0


tensor([[[[0.3059]]],


        [[[0.4495]]],


        [[[1.0000]]],


        [[[1.0000]]]], dtype=torch.float64)

In [7]:
import torch as th
# import torch
import numpy as np
from pytorch_lightning import seed_everything
seed_everything(0)

def _extract_into_tensor(arr, timesteps, broadcast_shape):
    """
    Extract values from a 1-D numpy array for a batch of indices.

    :param arr: the 1-D numpy array.
    :param timesteps: a tensor of indices into the array to extract.
    :param broadcast_shape: a larger shape of K dimensions with the batch
                            dimension equal to the length of timesteps.
    :return: a tensor of shape [batch_size, 1, ...] where the shape has K dims.
    """
    res = th.from_numpy(arr).to(device=timesteps.device)[timesteps].float()
    while len(res.shape) < len(broadcast_shape):
        res = res[..., None]
    return res.expand(broadcast_shape)

pred = th.randn(4, 3, 256, 256)
target = th.randn(4, 3, 256, 256)
bsz = 4

k = 5.0
num_timesteps = 1000
t = th.randint(0, num_timesteps, (bsz,)).long()

linear_start=1e-4
linear_end=2e-2
betas = (th.linspace(linear_start ** 0.5, linear_end ** 0.5, 1000, dtype=th.float64) ** 2).numpy()
alphas = 1. - betas
alphas_cumprod = np.cumprod(alphas, axis=0)
sqrt_alphas_cumprod = np.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = np.sqrt(1. - alphas_cumprod)

# snr = th.from_numpy(alphas_cumprod / (1 - alphas_cumprod))
alpha = _extract_into_tensor(sqrt_alphas_cumprod, t, t.shape)
sigma = _extract_into_tensor(sqrt_one_minus_alphas_cumprod, t, t.shape)
snr = (alpha / sigma) ** 2

mse_loss_weight = th.stack([snr, k * th.ones_like(t)], dim=1).min(dim=1)[0] / snr
mse_loss_weight


Global seed set to 0


tensor([0.3059, 0.4495, 1.0000, 1.0000])